In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import optuna
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from Klassifikator import get_objective, ResNet18, EarlyStopping

In [14]:
class TypeClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),  # EMNIST ist grau (1 Kanal)
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 3)  # 3 Typen
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

In [15]:
class ModularClassifier(nn.Module):
    def __init__(self, tm1, tm2, class_type_map):
        super().__init__()
        self.tm1 = tm1
        self.tm2 = tm2
        self.class_type_map = torch.tensor(class_type_map, dtype=torch.long)

    def forward(self, x):
        out_cls = self.tm1(x)  # (B, 36)
        out_type = self.tm2(x)  # (B, 3)

        # Umwandlung der Typklassenzuordnung pro Klasse (36 Klassen → 3 Typen)
        class_type_weights = out_type[:, self.class_type_map.to(x.device)]  # (B, 36)

        # Kombination der Outputs (elementweise Multiplikation)
        final_out = out_cls * class_type_weights

        return final_out, out_cls, out_type


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TypeClassifier(nn.Module):
    def __init__(self):
        super(TypeClassifier, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 3)  # 3 Typen: Großbuchstabe, Kleinbuchstabe, Zahl
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [17]:
class ModularClassifier(nn.Module):
    def __init__(self, tm1, tm2, class_type_map):
        super(ModularClassifier, self).__init__()
        self.tm1 = tm1  # z.B. ResNet18
        self.tm2 = tm2  # TypeClassifier
        self.class_type_map = torch.tensor(class_type_map, dtype=torch.long)

    def forward(self, x):
        out_cls = self.tm1(x)       # Shape: (B, 36)
        out_type = self.tm2(x)      # Shape: (B, 3)

        # class_type_map: (36,) → pro Klasse, welcher Typ (0, 1, 2)
        class_type_weights = out_type[:, self.class_type_map.to(x.device)]  # Shape: (B, 36)

        # Multipliziere Wahrscheinlichkeiten beider Module
        final_out = out_cls * class_type_weights

        return final_out, out_cls, out_type


In [18]:
from Klassifikator import ResNet18
from Datensatz import get_emnist_test_train, show_random_samples

In [19]:
class_list = list('0123456789ABCDEFGHIJKLMabcdefghijklm')
# 10 Zahlen, 13 Großbuchstaben, 13 Kleinbuchstaben
class_type_map = [2]*10 + [0]*13 + [1]*13


In [20]:
X_train, y_train, X_test, y_test,class_list = get_emnist_test_train()

Ziel-ASCII: [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109]
Anzahl Zielklassen: 36
⚠️ Klasse B: nur 3878 echte Bilder – augmentiere 2122 zusätzlich.
⚠️ Klasse D: nur 4562 echte Bilder – augmentiere 1438 zusätzlich.
⚠️ Klasse E: nur 4934 echte Bilder – augmentiere 1066 zusätzlich.
⚠️ Klasse G: nur 2517 echte Bilder – augmentiere 3483 zusätzlich.
⚠️ Klasse H: nur 3152 echte Bilder – augmentiere 2848 zusätzlich.
⚠️ Klasse J: nur 3762 echte Bilder – augmentiere 2238 zusätzlich.
⚠️ Klasse K: nur 2468 echte Bilder – augmentiere 3532 zusätzlich.
⚠️ Klasse L: nur 5076 echte Bilder – augmentiere 924 zusätzlich.
⚠️ Klasse b: nur 5159 echte Bilder – augmentiere 841 zusätzlich.
⚠️ Klasse c: nur 2854 echte Bilder – augmentiere 3146 zusätzlich.
⚠️ Klasse f: nur 2561 echte Bilder – augmentiere 3439 zusätzlich.
⚠️ Klasse g: nur 3687 echte Bilder – augmentiere 2313 zusätzlich.
⚠️ Klasse i: nur 272

In [21]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)


In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
# -----------------------------
# Optuna-Studie starten
# -----------------------------
study = optuna.create_study(direction="maximize")
study.optimize(get_objective(
          train_dataset=train_dataset,
          test_dataset=test_dataset,
          device=device,
          model=ResNet18(num_classes=len(class_list)).to(device),
          early_stopping=EarlyStopping(patience=4)), n_trials=20)

# Beste Parameter anzeigen
print("🎯 Beste Hyperparameter:")
for k, v in study.best_params.items():
    print(f"{k}: {v}")

[I 2025-06-22 16:11:11,579] A new study created in memory with name: no-name-9d01b06d-ac5a-4d42-96aa-d30ee8a978a8


📉 Epoch 1: Val Loss = 0.4245
📉 Epoch 2: Val Loss = 0.3959
📉 Epoch 3: Val Loss = 0.3810
📉 Epoch 4: Val Loss = 0.3432
📉 Epoch 5: Val Loss = 0.3443
📉 Epoch 6: Val Loss = 0.3442
📉 Epoch 7: Val Loss = 0.3330
📉 Epoch 8: Val Loss = 0.3496
📉 Epoch 9: Val Loss = 0.3533
📉 Epoch 10: Val Loss = 0.3539
⛔ Early Stopping in Epoch 11


[I 2025-06-22 16:20:16,195] Trial 0 finished with value: 0.8679722222222223 and parameters: {'batch_size': 128, 'lr': 0.040579030682126, 'momentum': 0.7425965082305384, 'step_size': 3, 'gamma': 0.6244501245492722}. Best is trial 0 with value: 0.8679722222222223.


⛔ Early Stopping in Epoch 1


[I 2025-06-22 16:21:48,748] Trial 1 finished with value: 0.8634722222222222 and parameters: {'batch_size': 64, 'lr': 0.009093430598994118, 'momentum': 0.7397987963092126, 'step_size': 4, 'gamma': 0.6603144242653538}. Best is trial 0 with value: 0.8679722222222223.


...

⛔ Early Stopping in Epoch 1


[I 2025-06-22 16:30:54,915] Trial 18 finished with value: 0.8681388888888889 and parameters: {'batch_size': 144, 'lr': 0.0010243564998505474, 'momentum': 0.801817883764402, 'step_size': 5, 'gamma': 0.867699233138312}. Best is trial 8 with value: 0.8715555555555555.


⛔ Early Stopping in Epoch 1


[I 2025-06-22 16:31:16,292] Trial 19 finished with value: 0.8691666666666666 and parameters: {'batch_size': 128, 'lr': 0.00021132125046572338, 'momentum': 0.642722878381176, 'step_size': 4, 'gamma': 0.7713114318292309}. Best is trial 8 with value: 0.8715555555555555.


🎯 Beste Hyperparameter:
batch_size: 256
lr: 0.001061419976981889
momentum: 0.7668524142972986
step_size: 5
gamma: 0.7181728886749222


In [26]:
# -----------------------------
# Finales Training mit besten Parametern 
# -----------------------------
best_params = study.best_params
train_loader = DataLoader(train_dataset, batch_size=best_params["batch_size"], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=best_params["batch_size"], shuffle=False)

model = ResNet18(num_classes=len(class_list)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=best_params["lr"], momentum=best_params["momentum"])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=best_params["step_size"], gamma=best_params["gamma"])


In [33]:
# ✅ Einheitlich definieren
class_list = list("0123456789ABCDEFGHIJKLMabcdefghijklm")  # = 36 Klassen
assert len(class_list) == len(set(class_list))  # doppelte vermeiden

# ✅ class_type_map definieren
class_type_map = []
for c in class_list:
    if c.isdigit():
        class_type_map.append(0)  # Ziffer
    elif c.isupper():
        class_type_map.append(1)  # Großbuchstabe
    else:
        class_type_map.append(2)  # Kleinbuchstabe

# ✅ type_labels passend zu den Klassen generieren
def get_type_label_tensor(label_tensor):
    type_labels = []
    for label in label_tensor:
        char = class_list[int(label)]
        if char.isdigit():
            type_labels.append(0)
        elif char.isupper():
            type_labels.append(1)
        else:
            type_labels.append(2)
    return torch.tensor(type_labels, dtype=torch.long)

# ✅ Dataloader vorbereiten
y_train_type = get_type_label_tensor(y_train)
y_test_type = get_type_label_tensor(y_test)

train_dataset = TensorDataset(X_train, y_train, y_train_type)
test_dataset = TensorDataset(X_test, y_test, y_test_type)

train_loader = DataLoader(train_dataset, batch_size=best_params["batch_size"], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=best_params["batch_size"], shuffle=False)

In [ ]:
# Initialisiere Modelle
tm1 = ResNet18(num_classes=len(class_list))  # Transfermodell aus Aufgabe 1.2

tm2 = TypeClassifier()

modular_model = ModularClassifier(tm1, tm2, class_type_map).to(device)

# Loss-Funktionen und Optimierer
criterion_cls = nn.CrossEntropyLoss()
criterion_type = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modular_model.parameters(), lr=1e-3)

# Training
for epoch in range(30):
    modular_model.train()
    total_loss = 0.0
    for images, labels_cls, labels_type in train_loader:
        images = images.to(device)
        labels_cls = labels_cls.to(device)
        labels_type = labels_type.to(device)

        final_out, out_cls, out_type = modular_model(images)

        loss_cls = criterion_cls(out_cls, labels_cls)
        loss_type = criterion_type(out_type, labels_type)
        loss = loss_cls + 0.5 * loss_type  # Gewichteter kombinierter Loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 590.9544
Epoch 2, Loss: 429.7407
Epoch 3, Loss: 389.3780
